In [2]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load chicago_table_id = "opendatadbt.311.chicago311" chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json? limit=10000& where=created_date >= '2021-03-18T00:00:00.000'" ) chicago311json = chicago311.json() chicago_df = pd.DataFrame(chicago311json) chicago_df = chicago_df[['created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']] chicago_df

In [51]:
#Incremental load
chicago_table_id = "opendatadbt.311.chicago311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.chicago311`
limit 10
"""

chicagomaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxopendatedate = str(chicagomaxopendatedataframe['f0_'][0])

chicagomaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxclosedate = str(chicagomaxclosedatedataframe['f1_'][0])


In [52]:
chicagomaxclosedate

'2021-04-05T06:13:24.000'

In [53]:
chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json?$limit=50000&$where=created_date > '" + chicagomaxopendatedate +"' OR closed_date between '" +chicagomaxclosedate+ "' AND '" + current_datetime + "'") 
chicago311json = chicago311.json()
cols = ['sr_number','created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']
chicago_df = pd.DataFrame(chicago311json, columns = cols)

chicago_df

,sr_number,created_date,closed_date,owner_department,sr_type,zip_code,street_address,city,status,latitude,longitude
0,SR21-00307906,2021-02-23T18:50:57.000,2021-04-05T08:19:54.000,DOB - Buildings,Building Violation,60640,5326 N WINTHROP AVE,NaN,Completed,41.979036338,-87.658282614
1,SR21-00306958,2021-02-23T15:42:33.000,2021-04-05T08:23:34.000,CDOT - Department of Transportation,Street Light Pole Damage Complaint,60640,5052 N SHERIDAN RD,NaN,Completed,41.974831483,-87.655387556
2,SR21-00124867,2021-01-22T17:32:30.000,2021-04-05T08:50:33.000,DOB - Buildings,No Building Permit and Construction Violation,60649,7542 S CREGIER AVE,NaN,Completed,41.757576848,-87.581299926
3,SR19-02901413,2019-11-07T09:30:28.000,2021-04-05T12:23:37.000,Streets and Sanitation,Tree Planting Request,60637,6034 S EBERHART AVE,NaN,Completed,41.784692436,-87.613557191
4,SR20-04363721,2020-06-16T13:45:27.000,2021-04-05T11:55:30.000,Streets and Sanitation,Tree Trim Request,60649,7618 S CHAPPEL AVE,NaN,Completed,41.75645334,-87.575130706
...,...,...,...,...,...,...,...,...,...,...,...
7876,SR21-00300663,2021-02-22T16:10:27.000,2021-04-05T09:00:16.000,CDOT - Department of Transportation,Snow – Uncleared Sidewalk Complaint,60613,830 W BUENA AVE,NaN,Completed,41.958730286,-87.65117702
7877,SR21-00301685,2021-02-22T19:45:26.000,2021-04-05T09:20:27.000,CDOT - Department of Transportation,Snow – Uncleared Sidewalk Complaint,60640,758 W GUNNISON ST,NaN,Completed,41.970125891,-87.65016848
7878,SR21-00301921,2021-02-22T21:13:35.000,2021-04-05T08:50:00.000,CDOT - Department of Transportation,Snow – Uncleared Sidewalk Complaint,60613,751 W BITTERSWEET PL,NaN,Completed,41.955642246,-87.649321201
7879,SR21-00301893,2021-02-22T21:03:13.000,2021-04-05T06:52:48.000,CDOT - Department of Transportation,Alley Light Out Complaint,60630,5047 W STRONG ST,NaN,Completed,41.970506583,-87.754511233


In [54]:
chicago_table_id = "opendatadbt.311.chicago311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('chicago311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    chicago_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [4]:
chicago_table_id = "opendatadbt.311.chicago311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "Chicago"
FROM `opendatadbt.311.chicago311`

"""

chicagodatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

chicagodatacount = chicagodatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

chicago_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    chicagodatacount, table_ref, job_config=job_config
).result()  # Make an API request.